In [1]:
import requests
import lxml.html as lh
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
url='https://osu.ppy.sh/rankings/osu/performance'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')


In [3]:
tr_elements = doc.xpath('//tr')
#Create empty list
col = []
i = 0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i += 1
    name = t.text_content().replace('\n                    ', '')
    name = name.replace('\n                ', '')
    col.append((name,[]))

col[0] = ("Rank", [])
col[1] = ("Username", [])

col

[('Rank', []),
 ('Username', []),
 ('Accuracy', []),
 ('Play Count', []),
 ('Performance', []),
 ('SS', []),
 ('S', []),
 ('A', [])]

In [4]:
for j in range(1,len(tr_elements)):
    #T is j'th row
    T=tr_elements[j]
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=float(data.replace(',', '.'))
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1
        

In [5]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

df = df.replace('\n                    ',value='' , regex=True)


In [6]:
df.head()

,Rank,Username,Accuracy,Play Count,Performance,SS,S,A
0,#1,...,98.66%,137.197,16.843,57.0,890.0,988.0
1,#2,...,99.22%,60.662,16.249,26.0,686.0,621.0
2,#3,...,98.07%,90.685,16.019,36.0,734.0,654.0
3,#4,...,98.09%,66.731,15.913,14.0,508.0,914.0
4,#5,...,97.89%,75.440,15.547,22.0,476.0,435.0


In [7]:
soup = BeautifulSoup(page.content, "html.parser")

In [8]:
names = soup.find_all('a', 
        class_="ranking-page-table__user-link-text js-usercard")

names_list = []
for name in names:
    names_list.append(name.get_text().split(sep='>')[-1][33:-29])

df['Username'] = names_list

In [9]:
countries = soup.find_all("span", class_="flag-country")

countries_list = []
for country in countries:
    countries_list.append(country.get('title'))

df.insert(loc=1, column='Country', value=countries_list)    

In [10]:
df.head()

,Rank,Country,Username,Accuracy,Play Count,Performance,SS,S,A
0,#1,United States,Vaxei,98.66%,137.197,16.843,57.0,890.0,988.0
1,#2,United States,idke,99.22%,60.662,16.249,26.0,686.0,621.0
2,#3,South Korea,FlyingTuna,98.07%,90.685,16.019,36.0,734.0,654.0
3,#4,United States,Freddie Benson,98.09%,66.731,15.913,14.0,508.0,914.0
4,#5,Russian Federation,Alumetri,97.89%,75.440,15.547,22.0,476.0,435.0


In [11]:
df.to_csv("./osu_ranking_top50")